In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [4]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312      1.0  66.811869                  NaN        NaN
0.000312      NaN        NaN                  1.0  29.223909
0.000750      2.0  30.192711                  2.0  25.037327
0.001052      3.0  28.255311                  3.0  26.636499
0.001476      4.0  25.341715                  4.0  27.109526
0.002070      5.0  22.174221                  5.0  26.136348
0.002904      6.0  19.226535                  6.0  23.684328
0.004074      7.0  16.722187                  7.0  20.092070
0.005714      8.0  14.724929                  8.0  16.181566
0.008015      9.0  13.199432                  9.0  12.952011
0.011243     10.0  12.055478                 10.0  11.083402
0.015771     11.0  11.189547                 11.0  10.510232
0.022122     12.0  10.518709                 12.0  10.598740
0.031031     13.0  10.089954                 13.0  10.837380
0.043528     14.0   9.446098                 14.0  10.506368
0.061057     15.0   8.347313                 15.0   9.289127
0.085645     16.0   7.340092                 16.0   7.796929
0.120136     17.0   6.662366                 17.0   6.559612
0.168516     18.0   6.248298                 18.0   5.851118
0.236378     19.0   6.084259                 19.0   5.766257
0.331549     20.0   6.196347                 20.0   6.163290
0.465100     21.0   6.809821                 21.0   7.026089
0.652400     22.0   6.510550                 22.0   6.804745
0.915100     23.0   5.072440                 23.0   5.222117
1.283650     24.0   3.910509                 24.0   3.710055
1.800600     25.0   3.126184                 25.0   2.786967
2.525700     26.0   2.505188                 26.0   2.385963
3.542800     27.0   1.954727                 27.0   2.152782
4.969550     28.0   1.571513                 28.0   1.805294
6.970850     29.0   1.401774                 29.0   1.476563
9.778100     30.0   1.262336                 30.0   1.257156
13.715850    31.0   1.074496                 31.0   1.202942
19.239350    32.0   0.841936                 32.0   1.061547
26.987250    33.0   0.698846                 33.0   0.833973
37.855300    34.0   0.648636                 34.0   0.655376
53.100050    35.0   0.551658                 35.0   0.570527
73.887500    36.0   0.434397                 36.0   0.522165
97.662500    37.0   0.339799                 37.0   0.405714
121.437500   38.0   0.247667                 38.0   0.249276
145.212500   39.0   0.170539                 39.0   0.125469
168.987500   40.0   0.126622                 40.0   0.067312
192.762500   41.0   0.095783                 41.0   0.043345
216.537500   42.0   0.070982                 42.0   0.033838
240.312500   43.0   0.047261                 43.0   0.026539
264.087500   44.0   0.012151                 44.0   0.004970
287.862500   45.0  -0.067821                 45.0  -0.071805
311.637500   46.0  -0.102537                 46.0  -0.103829
335.412500   47.0  -0.036092                 47.0  -0.028433
359.187500   48.0   0.019780                 48.0   0.027870
382.962500   49.0   0.036545                 49.0   0.039316
406.737500   50.0   0.044235                 50.0   0.043401
430.512500   51.0   0.048698                 51.0   0.046621
454.287500   52.0   0.051478                 52.0   0.048609
478.062500   53.0   0.053335                 53.0   0.050344
501.837500   54.0   0.054652                 54.0   0.051379
525.612500   55.0   0.055710                 55.0   0.052124
549.387500   56.0   0.056489                 56.0   0.052303
573.162500   57.0   0.056954                 57.0   0.052015
596.937500   58.0   0.056909                 58.0   0.051171
620.712500   59.0   0.056070                 59.0   0.049281
644.487500   60.0   0.054315                 60.0   0.046911
668.262500   61.0   0.051681                 61.0   0.043298
692.037500   62.0   0.048408                 62.0

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -14.290363   0.000000 -14.290363        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -14.027284   0.000188   
6.244000e-04 2     -14.291621   0.006200 -14.285421 -14.028477   0.003543   
8.759000e-04 3     -14.292224   0.007703 -14.284522 -14.028969   0.004781   
1.228600e-03 4     -14.292966   0.009625 -14.283341 -14.029593   0.006518   
1.723400e-03 5     -14.293870   0.012014 -14.281856 -14.030375   0.008888   
2.417400e-03 6     -14.294965   0.014932 -14.280033 -14.031344   0.012007   
3.390900e-03 7     -14.296284   0.018469 -14.277816 -14.032536   0.015930   
4.756500e-03 8     -14.297864   0.022754 -14.275111 -14.033988   0.020632   
6.672000e-03 9     -14.299741   0.027972 -14.271769 -14.035738   0.026054   
9.358900e-03 10    -14.301953   0.034385 -14.267568 -14.037824   0.032264   
1.312780e-02 11    -14.304530   0.042344 -14.262186 -14.040278   0.039666   
1.841450e-02 12    -14.307484   0.052305 -14.255179 -14.043110   0.049081   
2.583020e-02 13    -14.310774   0.064835 -14.245939 -14.046275   0.061557   
3.623230e-02 14    -14.314111   0.080606 -14.233506 -14.049480   0.078117   
5.082340e-02 15    -14.317192   0.100013 -14.217179 -14.052420   0.099218   
7.129060e-02 16    -14.320175   0.123234 -14.196941 -14.055217   0.124539   
1.000000e-01 17    -14.323171   0.151192 -14.171978 -14.057926   0.153767   
1.402710e-01 18    -14.325843   0.185646 -14.140196 -14.060157   0.187293   
1.967600e-01 19    -14.327477   0.229092 -14.098386 -14.061134   0.227427   
2.759970e-01 20    -14.326558   0.285280 -14.041278 -14.059303   0.279725   
3.871000e-01 21    -14.319311   0.359583 -13.959728 -14.051064   0.352609   
5.431000e-01 22    -14.292482   0.458595 -13.833887 -14.023792   0.455187   
7.617000e-01 23    -14.238761   0.573463 -13.665298 -13.971243   0.578864   
1.068500e+00 24    -14.170056   0.689104 -13.480952 -13.904588   0.702014   
1.498800e+00 25    -14.092345   0.810720 -13.281626 -13.829036   0.815591   
2.102400e+00 26    -14.004916   0.946815 -13.058101 -13.742947   0.928793   
2.949000e+00 27    -13.908836   1.101970 -12.806866 -13.647066   1.072215   
4.136600e+00 28    -13.814071   1.282194 -12.531876 -13.550719   1.278752   
5.802500e+00 29    -13.730627   1.508870 -12.221757 -13.464692   1.549015   
8.139200e+00 30    -13.651048   1.817300 -11.833748 -13.380752   1.873828   
1.141700e+01 31    -13.570396   2.226787 -11.343610 -13.293842   2.275095   
1.601470e+01 32    -13.503915   2.745508 -10.758406 -13.221170   2.857650   
2.246400e+01 33    -13.505078   3.389882 -10.115196 -13.221965   3.669514   
3.151050e+01 34    -13.601566   4.235268  -9.366298 -13.326140   4.667486   
4.420010e+01 35    -13.746123   5.354838  -8.391286 -13.484165   5.810756   
6.200000e+01 36    -13.916374   6.688273  -7.228101 -13.670933   7.200620   
8.577500e+01 37    -14.071263   8.066562  -6.004701 -13.837481   8.837901   
1.095500e+02 38    -14.143895   9.096176  -5.047719 -13.912763  10.055921   
1.333250e+02 39    -14.172398   9.822188  -4.350210 -13.938341  10.783612   
1.571000e+02 40    -14.205691  10.335774  -3.869917 -13.967203  11.165872   
1.808750e+02 41    -14.247556  10.734246  -3.513310 -14.002327  11.390588   
2.046500e+02 42    -14.300698  11.057142  -3.243556 -14.044771  11.555117   
2.284250e+02 43    -14.369705  11.326057  -3.043648 -14.096847  11.702502   
2.522000e+02 44    -14.464683  11.554138  -2.910545 -14.165223  11.845627   
2.759750e+02 45    -14.627340  11.751017  -2.876323 -14.293722  11.988125   
2.997500e+02 46    -15.017750  11.950421  -3.067329 -14.667709  12.159864   
3.235250e+02 47    -15.688954  12.332850  -3.356104 -15.354899  12.554608   
3.473000e+02 48    -16.469042  13.011291  -3.457751 -16.163739  13.283362   
3.710750e+02 49    -17

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')